# Titanic Disaster

Not much information is known about the titanic disaster. This not infamous disaster happened on April 15th, 1912 after the ship collided with an iceberg during her journey. In this project we are going to discover the underlying traits fro the features that enable people to survive the crash. Among the people survived there were kids, women, and men. We also discover what genger, age group survived the most and the underlying reasoning behind it. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

### Read Training and Test Data

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### Exploratory Data Analysis

Training and Test data have been copied to new variables to avoid the accidental data change. 

In [ ]:
train_e = train.copy()
test_e = test.copy()

Data Dive and quick look at the features

In [ ]:
train_e.info()

Passenger ID doesn't make any difference to the data training. Therefore we plan to drop it. Separating out numerical and cateogorical features.  

In [ ]:
train_e.drop(columns='PassengerId', axis=1, inplace=True)
test_e.drop(columns='PassengerId', axis=1, inplace=True)

In [ ]:
NUMERICAL_FEATURES =  ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']  
CATEGORICAL_FEATURES = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
TARGET = ['Survived']

There are five numerical and five categorical features and survived as the target variable. 

Taking a look at the data where the target variable has been removed from the test data.

In [ ]:
test_e.head()

In [ ]:
train_e.head()

Rearranging the Columns by moving the target variable to the end

In [ ]:
train_e = train_e[NUMERICAL_FEATURES + CATEGORICAL_FEATURES + TARGET]

Quick look on statistics of the numerical data

In [ ]:
train_e[NUMERICAL_FEATURES].describe()

In [ ]:
test_e[NUMERICAL_FEATURES].describe()

Quick dive into the categorical features

In [ ]:
train_e[CATEGORICAL_FEATURES].describe()

#### Findings:

* There are 891 records in the training data set.
* Age has only 714 values out of 891. 
* The range of age was from toddler to 80 year old. 
* Majority of the sample was in Pclass 3.
* There were a few people beyond the age of 38. 
* Majority of the family is a small family only a few family had more than one sibling. 
* Number of parents/children on were mostly 0 only a few family had 6 parents/children. 
* The fare varied form the range of 0 to $513. Most of the people paid less than 31 dollars.
* The names are unique all across, which means no two people had the same name. 
* There was no ambiguity in entering the data for Sex because there are only two types of values. 
* Cabin has similar information across many of their enteries this is due to the fact that a ship can have many cabins. 
* Embarked has 3 values which means that there are three stops from where people embarked. The value 'S' tops the world. 
* Out of 13 features in the training dataset. We have 6 integer values, 2 float data types, and 5 string data types

Checking the shape of the dataset

In [ ]:
print ("There are {} observation and {} columns in training dataset".format(train_e.shape[0], train_e.shape[1]))
print ("There are {} observation and {} columns in test dataset".format(test_e.shape[0], test_e.shape[1]))

#### Finding the missing columns:

In [ ]:
pd.DataFrame(train_e.isnull().sum(), columns= ['No. of missing values']).T

Age has 177 missing records wehreas Cabin has 687 missing records which is significant portion of the dataset. Embarked has only 2 missing values which is not that much. 

In [ ]:
pd.DataFrame(test_e.isnull().sum(), columns= ['No. of missing values']).T

Age has 86 missing records wehreas Cabin has 327 missing records, which is significant portion of the dataset. This similar kind of pattern was noticed in the training dataset as well.

#### Family Name 
Created a new feature Family value to distinguish people who belong to the same family. 

In [ ]:
train_e['FamilyName'] = train_e['Name'].str.split(",",expand=True)[0]
test_e['FamilyName'] = test_e['Name'].str.split(",",expand=True)[0]

In [ ]:
(train_e.FamilyName.value_counts() > 1).sum()

There are 133 families with more than one member on board.

There are 509 family name who does not have any clue about their cabin allocations. Cabin does not look significant feature This analysis is based on the assumption that people from the same family would share the same cabin information. The ticket number would be differnt but cabin information would definitely be same. Even if we try to fill in the missing data, we will still be missing 509 cabin information.

In [ ]:
cabin_yes = list(train_e[train_e['Cabin'].isnull() == False]['FamilyName'])
cabin_no = list(train_e[train_e['Cabin'].isnull() == True]['FamilyName'])

In [ ]:
len(list(set(cabin_no) - set(cabin_yes))) 

Therefore, we plan to carry forward our analysis by dropping the cabin information. 

In [ ]:
train_e.drop('Cabin', axis=1, inplace=True)
test_e.drop('Cabin', axis=1, inplace=True)

#### Survival by Age

In [ ]:
plot = sns.FacetGrid(train_e, col = 'Survived')
plot.map(plt.hist, 'Age', bins = 10)
plt.show()

Looking at the age data we can say that although most of the passengers are in their 15-40 age range but those are the ones that didn't survive. Mostly the people who survived are in the age range of 0-10 and elderly people. 

In [ ]:
grid = sns.FacetGrid(train_e, row='Pclass', col='Survived')
grid.map(plt.hist, 'Age', bins =10)
plt.show()

From the data visualization above, we can see that althougt Pclass 1 has the least amount of people those are the ones that survived the most. Pclass 3 has the most number of passengers but the survival rate is the lowest in proportion to the total number of passengers. 

In [ ]:
sns.set_style("darkgrid")
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(14,6))
sns.countplot(data = train_e[train_e.Survived > 0], x = "Pclass",ax=ax1)
ax1.set_title('Survived by Classes')
sns.countplot(data = train_e[train_e.Survived == 0], x = "Pclass",ax=ax2)
ax2.set_title('Not Survived By Classes')
sns.countplot(data = train_e, x = "Pclass",ax=ax3)
ax3.set_title('Passange By Classes')
plt.show()

In [ ]:
sns.set_style("darkgrid")
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(14,6))
sns.countplot(data = train_e[train_e.Survived > 0], x = "Embarked",ax=ax1)
ax1.set_title('Survived by Embarked')
sns.countplot(data = train_e[train_e.Survived == 0], x = "Embarked",ax=ax2)
ax2.set_title('Not Survived By Embarked')
sns.countplot(data = train_e, x = "Embarked",ax=ax3)
ax3.set_title('Passange By Embarked')
plt.show()

Most of the passengers embarked form port S, and those are the ones with the least survival rate. Port C had second highest number of passengers getting on board and those are the ones with the highest survival rate. 

In [ ]:
sns.set_style("darkgrid")
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(14,6))
sns.countplot(data = train_e[train_e.Survived > 0], x = "Sex", ax=ax1)
ax1.set_title('Survived by Gender')
sns.countplot(data = train_e[train_e.Survived == 0], x = "Sex", ax=ax2)
ax2.set_title('Not Survived By gender')
sns.countplot(data = train_e, x = "Sex",ax=ax3)
ax3.set_title('Passange By Gender')
plt.show()

From the data above we can see that although the population of the ship was male dominant but the survival rate was highly dependent on the gender. Females tend to have a higher survival rate as compared to the males. 

In [ ]:
grid = sns.FacetGrid(train_e, row='Sex', col='Survived')
g = grid.map(plt.hist, 'Age', bins =10)
plt.show()

From the data above we can see that females had a higher change of surival as compared to the males. 

In [ ]:
grid = sns.FacetGrid(train_e, row='Embarked', col='Survived')
g =  grid.map(plt.hist, 'Age', bins =10)
plt.show()

The data visualization above tells us that although most of the people embarked from C they didn't have as much as high chance of survival as compared to Embarked class Q. 

In [ ]:
train_e[train_e['Embarked'].isnull() == True]

Let's check other members of Icard and Stone family because these two are the only family that are missing Embarked values. 

In [ ]:
train_e[(train_e['FamilyName'].isin(['Icard','Stone']) == True) & (train_e['Embarked'].isnull() == False)].shape[0]

There are no other family members from Icard and Stone. Therefore, we decide to populate it with majority class. 'S' is the Embarked where most of the people boarded. Hence populatiing missing Embarked info with 'S'

In [ ]:
train_e['Embarked'].fillna('S',inplace = True)
test_e['Embarked'].fillna('S',inplace = True)

In [ ]:
sns.set_style("white")
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14,6))
sns.countplot(data = train_e[train_e.Survived > 0], x = "Embarked",ax=ax1,hue="Sex")
ax1.set_title('Survived by Embarked')
sns.countplot(data = train_e[train_e.Survived == 0], x = "Embarked",ax=ax2,hue="Sex")
ax2.set_title('Not Survived By Embarked')
sns.countplot(data = train_e, x = "Embarked",ax=ax3,hue="Sex")
ax3.set_title('Passange By Embarked')
plt.show()

In [ ]:
train_e['title'] = train_e['Name'].str.split(",",expand=True)[1].str.split(" ",expand=True)[1]
test_e['title'] = test['Name'].str.split(",",expand=True)[1].str.split(" ",expand=True)[1] 

In [ ]:
plt.figure(figsize=(18,5))
sns.set_style("white")
g =sns.countplot(data = train_e, x = "title")
g.set_title('Popular Title')
plt.show()

Different titles for the passengers on board

In [ ]:
test_e.loc[test_e['title'].isin(['Capt.','Dr.','Rev.','Major.','Col.','the','Don.', 'Jonkheer.', 'Sir.', 'Lady.']), 'title'] = 'Spl.Prof'
train_e.loc[train_e['title'].isin(['Capt.','Dr.','Rev.','Major.','Col.','the','Don.', 'Jonkheer.', 'Sir.', 'Lady.']), 'title'] = 'Spl.Prof'

Merging special titles into one title Special Profession

In [ ]:
train_e.loc[train_e['title'].isin(['Ms.','Mlle.']), 'title'] = 'Miss.'
train_e.loc[train_e['title'].isin(['Mme.']), 'title'] = 'Mrs.'

In [ ]:
test_e.loc[test_e['title'].isin(['Ms.','Mlle.']), 'title'] = 'Miss.'
test_e.loc[test_e['title'].isin(['Mme.']), 'title'] = 'Mrs.'

In [ ]:
plt.figure(figsize=(18,5))
sns.set_style("white")
g =sns.countplot(data = train_e, x = "title")
g.set_title('Popular Title')
plt.show()

All the different titles have been grouped to form one or more new title therefore maximizing the inter class similarity. 

Since it is harder to deal with categorical variables for the tiles, we will change them to numerical form.  We will also do the same for the sex.

In [ ]:
train_e[train_e.Age.isnull()]['title'].value_counts()

In [ ]:
test_e[test_e.Age.isnull()]['title'].value_counts()

In [ ]:
mr_mean_age = round(train_e[((train_e.title == 'Mr.') & ( train_e.Age != np.nan))]['Age'].mean())
miss_mean_age = round(train_e[((train_e.title == 'Miss.') & ( train_e.Age != np.nan))]['Age'].mean())
mrs_mean_age = round(train_e[((train_e.title == 'Mrs.') & ( train_e.Age != np.nan))]['Age'].mean())
master_mean_age = round(train_e[((train_e.title == 'Master.') & ( train_e.Age != np.nan))]['Age'].mean())
spl_mean_age = round(train_e[((train_e.title == 'Spl.Prof') & ( train_e.Age != np.nan))]['Age'].mean())

In [ ]:
print(mr_mean_age,miss_mean_age,mrs_mean_age,master_mean_age,spl_mean_age)

In [ ]:
train_e['Age'] = train_e.apply(
    lambda row: mr_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Mr.')) else row['Age'], axis = 1)
train_e['Age'] = train_e.apply(
    lambda row: mrs_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Mrs.')) else row['Age'], axis = 1)
train_e['Age'] = train_e.apply(
    lambda row: miss_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Miss.')) else row['Age'], axis = 1)
train_e['Age'] = train_e.apply(
    lambda row: master_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Master.')) else row['Age'], axis = 1)
train_e['Age'] = train_e.apply(
    lambda row: spl_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Spl.Prof')) else row['Age'], axis = 1)

In [ ]:
test_e['Age'] = test_e.apply(
    lambda row: mr_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Mr.')) else row['Age'], axis = 1)
test_e['Age'] = test_e.apply(
    lambda row: mrs_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Mrs.')) else row['Age'], axis = 1)
test_e['Age'] = test_e.apply(
    lambda row: miss_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Miss.')) else row['Age'], axis = 1)
test_e['Age'] = test_e.apply(
    lambda row: master_mean_age if ((np.isnan(row['Age'])) & (row['title'] == 'Master.')) else row['Age'], axis = 1)

In [ ]:
train_e.isnull().sum()
train_z = train_e.copy()

### Feature Engineering

##### Gender One Hot encoding

In [ ]:
train_e = train_z.copy()

In [ ]:
train_e['Sex'] = [1 if x == 'male' else 0 for x in train_e['Sex']]
test_e['Sex'] = [1 if x == 'male' else 0 for x in test_e['Sex']]
#test_z['Sex'] = [1 if x == 'male' else 0 for x in test_e['Sex']]

In [ ]:
train_e.isnull().sum()

In [ ]:
test_e.isnull().sum()

In [ ]:
title = {"Spl.Prof": 1, "Mr.": 2, "Miss.": 3, "Mrs.": 4, "Master.": 5}
train_e['title'] = train_e['title'].map(title)

test_e['title'] = test_e['title'].map(title)

In [ ]:
Embarked_label = {"S": 1, "C": 2, "Q": 3}
train_e['Embarked'] = train_e['Embarked'].map(Embarked_label)

test_e['Embarked'] = test_e['Embarked'].map(Embarked_label)

In [ ]:
train_e.head()

In [ ]:
test_e.head()

The way we decided to tackle the missing information of age is to fill in values with the median values of similar pclass and SibSp. We decided to choose those two features because from the coorelation table below we can see that Pclass and the SibSb have the largeset coorelation with the Age.  

In [ ]:
train_Name = train_e.pop('Name')
train_ticket = train_e.pop('Ticket')
train_FamilyName = train_e.pop('FamilyName')

In [ ]:
test_Name = test_e.pop('Name')
test_ticket = test_e.pop('Ticket')
test_FamilyName = test_e.pop('FamilyName')

In [ ]:
train_e.head(10)

In [ ]:
train_e['family'] = train_e['SibSp'] + train_e['Parch']
test_e['family'] = test_e['SibSp'] + test_e['Parch']

In [ ]:
test_e['family'] = test_e.apply(
    lambda row: 1 if (row['family'] > 0 ) else 0, axis = 1)

In [ ]:
train_e['family'] = train_e.apply(
    lambda row: 1 if (row['family'] > 0 ) else 0, axis = 1)

In [ ]:
train_SibSp = train_e.pop('SibSp')
train_Parch = train_e.pop('Parch')

test_SibSp = test_e.pop('SibSp')
test_Parch = test_e.pop('Parch')

In [ ]:
train_e.head()

In [ ]:
s = pd.Series(np.array([0, 5, 12, 18, 21, 30, 40, 50, 60, 70, 80]),
    index=['toddler','toddler','Young', 'Teen', 'Adult', '30s','40s','50s','60s','70s','80s'])

In [ ]:
train_e['Age_Bucket'] = pd.cut(train_e['Age'],s)
test_e['Age_Bucket'] = pd.cut(test_e['Age'],s)

In [ ]:
test_e.head()

In [ ]:
train_e = pd.concat([train_e, pd.get_dummies(train_e['Pclass'], prefix='Pclass',drop_first=True)],axis = 1)
train_e = pd.concat([train_e, pd.get_dummies(train_e['Embarked'], prefix='Embarked',drop_first=True)],axis = 1)
train_e = pd.concat([train_e, pd.get_dummies(train_e['title'], prefix='title',drop_first=True)],axis = 1)
train_e = pd.concat([train_e, pd.get_dummies(train_e['Age_Bucket'], prefix='Age_Bucket',drop_first=True)],axis = 1)
train_e = pd.concat([train_e, pd.get_dummies(train_e['Sex'], prefix='Sex',drop_first=True)],axis = 1)

In [ ]:
test_e = pd.concat([test_e, pd.get_dummies(train_e['Pclass'], prefix='Pclass',drop_first=True)],axis = 1)
test_e = pd.concat([test_e, pd.get_dummies(train_e['Embarked'], prefix='Embarked',drop_first=True)],axis = 1)
test_e = pd.concat([test_e, pd.get_dummies(train_e['title'], prefix='title',drop_first=True)],axis = 1)
test_e = pd.concat([test_e, pd.get_dummies(train_e['Age_Bucket'], prefix='Age_Bucket',drop_first=True)],axis = 1)
test_e = pd.concat([test_e, pd.get_dummies(train_e['Sex'], prefix='Sex',drop_first=True)],axis = 1)

In [ ]:
train_e.head(2).T

In [ ]:
train_e.pop('Pclass')
train_e.pop('Age')
train_e.pop('Sex')
train_e.pop('Embarked')
train_e.pop('title')
Age_Bucket_train = train_e.pop('Age_Bucket')

In [ ]:
test_e.pop('Pclass')
test_e.pop('Age')
test_e.pop('Sex')
test_e.pop('Embarked')
test_e.pop('title')
Age_Bucket_test = test_e.pop('Age_Bucket')

In [ ]:
test_e.head(3)

In [ ]:
train_e.to_csv('clean_train.csv', sep=',')
y = train_e.pop('Survived')
X = train_e

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.05)

In [ ]:
def model_summary(model_obj, model_name):
    model_obj.fit(X_train, y_train)
    print('-'*40)
    print('Model : {}'.format(model_name))
    print('-'*40)
    print('Traing dataset score: {}'.format(model_obj.score(X_train, y_train)))
    model_pred = model_obj.predict(X_test)
    print('Test Data Metrics')
    print ('Accuracy Score :\t{:.4}'.format(accuracy_score(y_test,model_pred)))
    print ('Recall Score :\t\t{:.4}'.format(recall_score(y_test,model_pred)))
    print ('Precision Score :\t{:.4}'.format(precision_score(y_test,model_pred)))
    print(confusion_matrix(y_test, model_pred))
    print(cross_val_score(model_obj, X, y, cv=5))  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

logit = LogisticRegression(solver='newton-cg', random_state=42, max_iter=1000,)
rf = RandomForestClassifier(n_estimators=30, random_state=42)
linear_svc = SVC(kernel='linear')
rbf_svc = SVC(kernel='rbf')
poly_svc = SVC(kernel='poly')
gvc = GradientBoostingClassifier()


model_summary(logit, 'LogisticRegression')
model_summary(rf, 'Random Forest')
model_summary(linear_svc, 'Linear SVM')
model_summary(rbf_svc, 'Radial SVM')
model_summary(poly_svc, 'Ploynominal SVM')
model_summary(gvc, 'GradientBoostingClassifier')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([('classifier' , LogisticRegression())])

param_grid = [
    {'classifier' : [LogisticRegression()],
    # 'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : [0.001,0.01,0.1,10,100,1000],
    'classifier__solver' : ['liblinear' , 'newton-cg', 'sag' , 'lbfgs' ]}
     #'max_iter': [100,300,500,700, 900, 1000]}   
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)

In [ ]:
best_clf

In [ ]:
best_clf.score(X_train, y_train)

In [ ]:
model_pred = best_clf.predict(X_test)
print('Test Data Metrics')
print ('Accuracy Score :\t{:.4}'.format(accuracy_score(y_test,model_pred)))
print ('Recall Score :\t\t{:.4}'.format(recall_score(y_test,model_pred)))
print ('Precision Score :\t{:.4}'.format(precision_score(y_test,model_pred)))
print(confusion_matrix(y_test, model_pred))

In [ ]:
train_e[train_e.columns[0:]].corr()['Age'][:]

In [ ]:
# age_matrix = np.zeros((8,3))

# for i in range(0, 8):
#     for j in range(0, 3):
#         expected_value = train_e[(train_e['SibSp'] == i) & (train_e['Pclass'] == j+1)]['Age'].dropna()
#         age = expected_value.median()
#         age_matrix[i,j] = float(age/0.5 + 0.5 ) * 0.5
    
# for i in range(0, 8):
#     for j in range(0, 3):
#         train_e.loc[(train_e.Age.isnull()) & (train_e.SibSp == i) & (train_e.Pclass == j+1), 'Age'] = age_matrix[i,j]       
# train_e['Age'] = train_e['Age'].astype(float)

In [ ]:
# test_fare_median = test_e['Fare'].median()
# test_e['Fare'].fillna(test_fare_median, inplace=True)